In [14]:
from captcha.image import ImageCaptcha
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
 
text = '1234'
image = ImageCaptcha()
captcha = image.generate(text)
captcha_image = Image.open(captcha)
#captcha_image.show()
plt.imshow(captcha_image)

ModuleNotFoundError: No module named 'captcha.image'; 'captcha' is not a package

In [ ]:
VOCAB = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
CAPTCHA_LENGTH = 4
VOCAB_LENGTH = len(VOCAB)

In [6]:
from captcha.image import ImageCaptcha
import numpy as np

ModuleNotFoundError: No module named 'captcha.image'; 'captcha' is not a package

In [ ]:
def generate_captcha(captcha_text):
    '''
    get captcha text
    return np array of captcha image
    '''
    image = ImageCaptcha()
    captcha = image.generate(captcha_text)
    captcha_image = Image.open(captcha)
    captcha_array = np.array(captcha_image)
    return captcha_array

In [ ]:
captcha = '1234'
generate_captcha(captcha)

In [ ]:
def text2vec(text):
    '''
    text to vector
    :param text: source text
    :return: np array
    '''
    if len(text) > CAPTCHA_LENGTH:
        return False
    
    vector = np.zeros(CAPTCHA_LENGTH * VOCAB_LENGTH)
    
    for i, c in enumerate(text):
        index = i * VOCAB_LENGTH + VOCAB.index(c)
        vector[index] = 1
    return vector

def vec2text(vector):
    '''
    vector to text
    :param text: source vector
    :return: text'''
    if len(vector) != 40:
        return False
    
    if not isinstance(vector, np.ndarray):
        vector = np.asarray(vector)
    
    text = ''
    
    for i in range(0,4):
        text += VOCAB[np.argmax(vector[i*10:(i+1)*10])]
    
    return text
    

In [ ]:
text = '2333'
vec_ = text2vec(text)
text_ = vec2text(vec_)
vec_, text_

In [ ]:
################## pickle功能介绍 ######################
import pickle
data = ['1234', '2333']
p_str = pickle.dumps(data)
print(p_str)

In [ ]:
mes = pickle.loads(p_str)
print(mes)

In [ ]:
# print(type(data))
with open('.\tmp.pk', 'wb+') as f:
    pickle.dump(data, f)

In [ ]:
with open('.\tmp.pk', 'rb+') as f:
    data = pickle.load(f)
data

In [ ]:
#########################################################

In [ ]:
import random
from os.path import join, exists
import pickle
import numpy as np
from os import makedirs

DATA_LENGTH = 1000
DATA_PATH = 'data'

def get_random_text():
    text = ''
    for i in range(CAPTCHA_LENGTH):
        text += random.choice(VOCAB)
    return text



In [ ]:
print(get_random_text())

In [ ]:
def generate_data():
    print('Generating Data...')
    data_x, data_y = [], []

    # generate data x and y
    for i in range(DATA_LENGTH):
        text = get_random_text()
        # get captcha array
        captcha_array = generate_captcha(text)
        # get vector
        vector = text2vec(text)
        data_x.append(captcha_array)
        data_y.append(vector)

    # write data to pickle
    if not exists(DATA_PATH):
        makedirs(DATA_PATH)

    x = np.asarray(data_x, np.float32)
    y = np.asarray(data_y, np.float32)
    with open(join(DATA_PATH, 'data.pkl'), 'wb') as f:
        pickle.dump(x, f)
        pickle.dump(y, f)
        


In [ ]:
generate_data()
with open(join(DATA_PATH,'data.pkl'), 'rb') as f:
    data_x = pickle.load(f)
    data_y = pickle.load(f)
#     return standardize(data_x), data_y


In [ ]:
print(data_x.shape, data_y.shape, type(data_x))

In [ ]:
from sklearn.cross_validation import train_test_split
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.4, random_state=40)

In [ ]:
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape, type(train_x))

In [ ]:
train_x[1]

In [ ]:
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch

In [ ]:
def image_grayscale_deal(image):
    """
    图片转灰度处理
    :param image:图片文件
    :return: 转灰度处理后的图片文件
    """
    image = image.convert('L')
	#取消注释后可以看到处理后的图片效果
    plt.subplot(1,3,2)
    plt.imshow(image)
    # image.show()
    return image

def image_thresholding_method(image):
    """
    图片二值化处理
    :param image:转灰度处理后的图片文件
    :return: 二值化处理后的图片文件
    """
    # 阈值
    threshold = 160
    table = []
    for i in range(256):
        if i < threshold:
            table.append(0)
        else:
            table.append(1)
    # 图片二值化，此处第二个参数为数字一
    image = image.point(table, '1')
	#取消注释后可以看到处理后的图片效果
    plt.subplot(1,3,3)
    plt.imshow(image)
    #image.show()
    return image


def captcha_tesserocr_crack(image):
    """
    图像识别
    :param image: 二值化处理后的图片文件
    :return: 识别结果
    """
    result = tesserocr.image_to_text(image)
    return result


In [ ]:
class captcha(Dataset):
    def __init__(self, x, y, transform=None):
        super().__init__()
        self.x = x
        self.y = y
        self.transform = transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, n):
        return self.x[n], self.y[n]

In [ ]:
from torchvision import transforms

trans = transforms.Compose(transforms = [
    transforms.ToTensor()
])